In [1]:
import numpy as np
import scipy as sp

We seek to find the FFT of a bursty signal that begins at times $x(t_0), x(t_1) ...$ and lasts for $t_L$ seconds.
The most obvious way to perform this is to capture all the bursts in one window and perform a single FFT.
However, this can be very inefficient if the gap between each burst is very large i.e. $t_0 + t_L << t_1, t_1 + t_L << t_2 ...$. In such a scenario, zeroing out the empty space between each burst is commonplace and will result in the FFT being needlessly computed over all the '0' samples.

Instead, we would like to perform the FFT over each short burst and then combine them such that the result for a particular bin would be the same as if the FFT had been taken over all the bursts together.

To do this, consider the standard form of the FFT:

$$X(k) = \sum^{N-1}_{n} x(n) e^{-i 2 \pi k n / N}, k = 0, 1, ... N-1 $$

In practice, one would have pre-processed the bursts to at least know the samples at which each burst starts; that is, we can define the samples $n_0, n_1 ...$ as the starting samples for each burst. Similarly, we can define the length of the burst as $L$ (this can be rounded up to the nearest sample if $t_L$ sits in a subsample range.

We would now perform the FFTs over each burst in the following way:

$$X_i (k) = \sum^{L-1}_{l} x(n_i + l) e^{-i 2 \pi k l / L}, k = 0, 1, .. L-1 $$

Now, clearly the FFT resolution for the short burst is far worse i.e. the bin widths are larger due to taking a smaller window. We will get to this. For now, suppose the original (large) window length is an integer multiple of the shorter window length i.e. $N = \alpha L$.

Then, every $\alpha$'th bin in the original (large) window corresponds to a bin in the shorter window. We will use this to demonstrate the corrections required.

Consider the first pair of bins which correspond to each other: this is the integer $\alpha$ in the original (large) window and the integer $1$ in the shorter window. There is no change in the 0'th terms (which do pair up with each other) as they are simply summations over all samples, and are equivalent without any corrections.

Remembering that $N = \alpha L$, we have

$$X(\alpha) = \sum^{N-1}_{n} x(n) e^{-i 2 \pi \alpha n / N}, k = 0, 1, ... N-1 $$
$$X_i (1) = \sum^{L-1}_{l} x(n_i + l) e^{-i 2 \pi \alpha l / N}, k = 0, 1, .. L-1 $$

Consider the term at the start of the burst - the sample at $x(n_i + l)$. The contribution of this term to the original window is given by 

$$F_o = x(n_i + l) e^{-i 2 \pi \alpha (n_i + l) / N}$$

whereas the term from the short window is given by

$$F_s = x(n_i + l) e^{-i 2 \pi \alpha (l) / N}$$

which clearly shows that 

$$ F_0 = F_s e^{-i 2 \pi \alpha (n_i) / N} $$